In [1]:
import os
import glob
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import (preprocess_input)
from tensorflow.keras.callbacks import EarlyStopping, Callback
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils import class_weight
import matplotlib.pyplot as plt

In [2]:
# === CONFIG ===
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)

BASE_PATH = r"C:\Users\ADITYA DAS\Desktop\Machine Learning\CP_DATASETS\new\CP_DATASET_GAN"
CLASSES = ["BLIGHT", "BLAST", "BROWNSPOT", "HEALTHY"]
IMG_SIZE = (224, 224)
BATCH_SIZE = 32  # ✅ Smaller for BN layers
EPOCHS = 30
LEARNING_RATE = 1e-5  # ✅ Lower for frozen backbone

In [3]:
# === Load filepaths & labels ===
all_filepaths, all_labels = [], []
for idx, class_name in enumerate(CLASSES):
    aug_path = os.path.join(BASE_PATH, class_name, "augmented")
    files = glob.glob(os.path.join(aug_path, "*.jpg")) + \
            glob.glob(os.path.join(aug_path, "*.jpeg")) + \
            glob.glob(os.path.join(aug_path, "*.png"))
    all_filepaths.extend(files)
    all_labels.extend([idx] * len(files))

print(f"✅ Total images found: {len(all_filepaths)}")

✅ Total images found: 32460


In [4]:
# === tf.data.Dataset ===
filepaths_ds = tf.data.Dataset.from_tensor_slices(all_filepaths)
labels_ds = tf.data.Dataset.from_tensor_slices(all_labels)
ds = tf.data.Dataset.zip((filepaths_ds, labels_ds)).shuffle(len(all_filepaths), seed=SEED)

train_size = int(0.8 * len(all_filepaths))
train_ds = ds.take(train_size)
val_ds = ds.skip(train_size)

print(f"✅ Train samples: {train_size} | Val samples: {len(all_filepaths) - train_size}")

✅ Train samples: 25968 | Val samples: 6492


In [5]:
# === Image Processor ===
def process_img(filepath, label):
    img = tf.io.read_file(filepath)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)

    img = preprocess_input(img)  # ✅ For ResNet50: scale to [-1, 1] the same way as pretraining

    label = tf.one_hot(label, depth=len(CLASSES))
    return img, label

In [6]:
# === Final Pipeline ===
train_ds = train_ds.map(process_img).batch(BATCH_SIZE)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(process_img).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [7]:
# === DenseNet50 Model ===
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# ✅ Unfreeze last 30 layers
for layer in base_model.layers:    #[:-30]:
    layer.trainable = False
# for layer in base_model.layers[-30:]:
#     layer.trainable = True

inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)  # Important: BN layers training mode ON for partial unfrozen!
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
outputs = Dense(len(CLASSES), activation='softmax')(x)
model = Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

In [8]:
# === Learning rate logger ===
class LearningRateLogger(Callback):
    def on_epoch_end(self, epoch, logs=None):
        lr = self.model.optimizer.lr
        if hasattr(lr, '__call__'):
            lr = lr(self.model.optimizer.iterations)
        if hasattr(lr, 'numpy'):
            lr = lr.numpy()
        print(f"📉 Learning rate at epoch {epoch+1}: {lr:.6f}")

# === Compute class weights ===
# y_train_int = np.argmax(np.concatenate([labels.numpy() for _, labels in train_ds.unbatch().batch(BATCH_SIZE)]), axis=1)
# class_weights = dict(enumerate(class_weight.compute_class_weight(
#     class_weight='balanced',
#     classes=np.arange(len(CLASSES)),
#     y=y_train_int
# )))
# print("✅ Computed class weights:", class_weights)

In [9]:
# === Train ===
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[EarlyStopping(patience=4, restore_best_weights=True), LearningRateLogger()]
    # class_weight=class_weights
)

Epoch 1/30
📉 Learning rate at epoch 1: 0.000010==] - ETA: 0s - loss: 1.2067 - accuracy: 0.4865     
812/812 [==============================] - 194s 228ms/step - loss: 1.2067 - accuracy: 0.4865 - val_loss: 0.9568 - val_accuracy: 0.6696
Epoch 2/30
📉 Learning rate at epoch 2: 0.000010==] - ETA: 0s - loss: 0.9863 - accuracy: 0.6374  
812/812 [==============================] - 184s 227ms/step - loss: 0.9863 - accuracy: 0.6374 - val_loss: 0.8484 - val_accuracy: 0.7287
Epoch 3/30
📉 Learning rate at epoch 3: 0.000010==] - ETA: 0s - loss: 0.9032 - accuracy: 0.6903  
812/812 [==============================] - 184s 227ms/step - loss: 0.9032 - accuracy: 0.6903 - val_loss: 0.7978 - val_accuracy: 0.7631
Epoch 4/30
📉 Learning rate at epoch 4: 0.000010==] - ETA: 0s - loss: 0.8540 - accuracy: 0.7226  
812/812 [==============================] - 182s 224ms/step - loss: 0.8540 - accuracy: 0.7226 - val_loss: 0.7573 - val_accuracy: 0.7796
Epoch 5/30
📉 Learning rate at epoch 5: 0.000010==] - ETA: 0s - loss: 

1/1 [==============================] - 1s 998ms/step

📊 Classification Report:
              precision    recall  f1-score   support

      BLIGHT       0.98      0.97      0.97      1615
       BLAST       0.83      0.77      0.80      1618
   BROWNSPOT       0.90      0.88      0.89      1636
     HEALTHY       0.82      0.91      0.86      1623

    accuracy                           0.88      6492
   macro avg       0.88      0.88      0.88      6492
weighted avg       0.88      0.88      0.88      6492



✅ Model saved at: C:\Users\ADITYA DAS\Desktop\Machine Learning\NEW_STAGE_MODELS\gan\DenseNet121_Phase1_gan_noaug.h5


In [10]:
# === Evaluate ===
y_true, y_pred = [], []
for images, labels in val_ds:
    preds = model.predict(images)
    y_pred.extend(np.argmax(preds, axis=1))
    y_true.extend(np.argmax(labels.numpy(), axis=1))

print("\n📊 Classification Report:")
print(classification_report(y_true, y_pred, target_names=CLASSES))

cm = confusion_matrix(y_true, y_pred)
ConfusionMatrixDisplay(cm, display_labels=CLASSES).plot(cmap="Blues", xticks_rotation=45)
plt.title("Confusion Matrix")
plt.tight_layout()
plt.show()

# === Save ===
SAVE_PATH = r"C:\Users\ADITYA DAS\Desktop\Machine Learning\NEW_STAGE_MODELS\gan\DenseNet121_Phase1_gan_noaug.h5"
model.save(SAVE_PATH)
print(f"✅ Model saved at: {SAVE_PATH}")
